In [374]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Monthly Unemployment:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, monthly  
+ Select the following dimensions: Regions, Year/Month   
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)   
+ Under "Year" choose your range (most recently I've selected 1990 - recent just to get it all)   
+ Select the value "Unemployment Rate (Seasonally Adjusted), Labor Force (Seasonally Adjusted), Employed (Seasonally Adjusted)"  
+ Swap rows & columns if needed so "Regions" data is at the top, and "Year/Month" is at the side.  

When you've downloaded this csv, open it and delete the top line, replace "region" with "NAME" as a double header, unmerge the time and make sure it's copy-pasted across the relevant columns and keep the headers for different datapoints. Delete the data notes at the bottom, and select all of the numerical data and make them "General" format. Save as: 'JobsEQ_LAUSMonthly_GeosExceptPlaces.csv'  

In [404]:
#import monthly employment data and examine, use dtype str to import dates correctly
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSMonthly_GeosExceptPlaces.csv', dtype = str)
data.head(2)

,Time,USA,USA.1,USA.2,USA.3,Tennessee (47),Tennessee (47).1,Tennessee (47).2,Tennessee (47).3,GNRC,GNRC.1,GNRC.2,GNRC.3,MPO,MPO.1,MPO.2,MPO.3,GNRC Region,GNRC Region.1,GNRC Region.2,GNRC Region.3,"Cheatham County, Tennessee (47021)","Cheatham County, Tennessee (47021).1","Cheatham County, Tennessee (47021).2","Cheatham County, Tennessee (47021).3","Davidson County, Tennessee (47037)","Davidson County, Tennessee (47037).1","Davidson County, Tennessee (47037).2","Davidson County, Tennessee (47037).3","Dickson County, Tennessee (47043)","Dickson County, Tennessee (47043).1","Dickson County, Tennessee (47043).2","Dickson County, Tennessee (47043).3","Houston County, Tennessee (47083)","Houston County, Tennessee (47083).1","Houston County, Tennessee (47083).2","Houston County, Tennessee (47083).3","Humphreys County, Tennessee (47085)","Humphreys County, Tennessee (47085).1","Humphreys County, Tennessee (47085).2","Humphreys County, Tennessee (47085).3","Maury County, Tennessee (47119)","Maury County, Tennessee (47119).1","Maury County, Tennessee (47119).2","Maury County, Tennessee (47119).3","Montgomery County, Tennessee (47125)","Montgomery County, Tennessee (47125).1","Montgomery County, Tennessee (47125).2","Montgomery County, Tennessee (47125).3","Robertson County, Tennessee (47147)","Robertson County, Tennessee (47147).1","Robertson County, Tennessee (47147).2","Robertson County, Tennessee (47147).3","Rutherford County, Tennessee (47149)","Rutherford County, Tennessee (47149).1","Rutherford County, Tennessee (47149).2","Rutherford County, Tennessee (47149).3","Stewart County, Tennessee (47161)","Stewart County, Tennessee (47161).1","Stewart County, Tennessee (47161).2","Stewart County, Tennessee (47161).3","Sumner County, Tennessee (47165)","Sumner County, Tennessee (47165).1","Sumner County, Tennessee (47165).2","Sumner County, Tennessee (47165).3","Trousdale County, Tennessee (47169)","Trousdale County, Tennessee (47169).1","Trousdale County, Tennessee (47169).2","Trousdale County, Tennessee (47169).3","Williamson County, Tennessee (47187)","Williamson County, Tennessee (47187).1","Williamson County, Tennessee (47187).2","Williamson County, Tennessee (47187).3","Wilson County, Tennessee (47189)","Wilson County, Tennessee (47189).1","Wilson County, Tennessee (47189).2","Wilson County, Tennessee (47189).3"
0,Time,Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Unemployment Rate (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unem

In [405]:
#set index to the extraneous "Time" header and transpose, then rename to "Datapoint" and reset index without dropping
data = data.set_index('Time').transpose()
data = data.rename(columns = {'Time': 'Datapoint'}).reset_index(drop = False)

In [406]:
#make sure there's no head or tail on the strings
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [407]:
#split and expand the geography names on the "." (where there were multiple from duplicates because of how we imported it) and take what was before as the name
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [408]:
#drop the original column, set the index to NAME and transpose to rename
data = data.drop(columns = 'index').set_index('NAME').transpose()

In [409]:
#rename, and transpose back and reset the index without dropping
data = data.rename(columns = {'USA': 'US', 'Tennessee (47)': 'Tennessee', 'Cheatham County, Tennessee (47021)': 'Cheatham County', 
                              'Davidson County, Tennessee (47037)': 'Davidson County', 'Dickson County, Tennessee (47043)': 'Dickson County', 
                              'Houston County, Tennessee (47083)': 'Houston County', 'Humphreys County, Tennessee (47085)': 'Humphreys County', 
                              'Maury County, Tennessee (47119)': 'Maury County', 'Montgomery County, Tennessee (47125)': 'Montgomery County', 
                              'Robertson County, Tennessee (47147)': 'Robertson County', 'Rutherford County, Tennessee (47149)': 'Rutherford County', 
                              'Stewart County, Tennessee (47161)': 'Stewart County', 'Sumner County, Tennessee (47165)': 'Sumner County', 
                              'Trousdale County, Tennessee (47169)': 'Trousdale County', 'Williamson County, Tennessee (47187)': 'Williamson County', 
                              'Wilson County, Tennessee (47189)': 'Wilson County'
                             })
data = data.transpose().reset_index(drop = False)

In [410]:
#go from wide to long format with .melt() naming the columns as single column "Time", keeping "Datapoint" and "NAME" as is, and calling the number "Value"
cols = data.columns
data = data.melt(var_name = 'Time', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head(2)

,NAME,Datapoint,Time,Value
0,US,Labor Force (Seas Adj),Jan-90,126027143.4
1,US,Employed (Seas Adj),Jan-90,118985218.4


In [411]:
#set index and transpose on datapoint to rename the original categories from JobsEQ, then transpose and reset index again without dropping
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force (Seas Adj)': 'LF:Labor Force', 'Employed (Seas Adj)': 'LF:Employed', 
                              'Unemployed (Seas Adj)': 'LF:Unemployed', 'Unemployment Rate (Seas Adj)': 'LF:Unemployed %'})
data = data.transpose().reset_index(drop = False)

In [412]:
#make sure the value is a float datatype
data['Value'] = data['Value'].astype(float)

In [413]:
#create a new column equal to the value
data['New Value'] = data['Value']

In [414]:
#use .loc to say that in instances where the datapoint is equal to unemployment rate, new value is equal to value times 100 to match percent formatting
data.loc[data['Datapoint'] == 'LF:Unemployed %', 'New Value'] = data['Value'] * 100

In [415]:
#data['month'].value_counts()

In [416]:
data = data.drop(columns = 'Value').rename(columns = {'New Value': 'Value'})
data.head()

,Datapoint,NAME,Time,Value
0,LF:Labor Force,US,Jan-90,1.260271e+08
1,LF:Employed,US,Jan-90,1.189852e+08
2,LF:Unemployed,US,Jan-90,7.090043e+06
3,LF:Unemployed %,US,Jan-90,5.514683e+00
4,LF:Labor Force,Tennessee,Jan-90,2.372424e+06


In [417]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
weirds = ['Jan-00', 'Feb-00', 'Mar-00', 'Apr-00', 'May-00', 'Jun-00', 'Jul-00', 'Aug-00', 'Sep-00', 'Oct-00', 'Nov-00', 'Dec-00']
twothousand = data.loc[data['Time'].isin(weirds)]
data = data.loc[~data['Time'].isin(weirds)]

In [418]:
data.tail()

,Datapoint,NAME,Time,Value
30091,LF:Unemployed %,Williamson County,December-22,2.443179
30092,LF:Labor Force,Wilson County,December-22,81272.654230
30093,LF:Employed,Wilson County,December-22,78996.896800
30094,LF:Unemployed,Wilson County,December-22,2146.206382
30095,LF:Unemployed %,Wilson County,December-22,2.718806


In [419]:
#format the time for datetime by splitting the string and creating a month and year column
boop = data['Time'].str.split(pat = "-", expand = True)
data['month'] = boop[0]
data['year'] = boop[1]

In [420]:
twothousand.head()

,Datapoint,NAME,Time,Value
9424,LF:Labor Force,US,May-00,1.438651e+08
9425,LF:Employed,US,May-00,1.382064e+08
9426,LF:Unemployed,US,May-00,5.868334e+06
9427,LF:Unemployed %,US,May-00,4.058950e+00
9428,LF:Labor Force,Tennessee,May-00,2.852116e+06


In [421]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
boop = twothousand['Time'].str.split(pat = "-", expand = True)
twothousand['month'] = boop[0]
twothousand['year'] = boop[1]

In [422]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
df = data.append(twothousand)
data = df

C:\Users\jmccall\AppData\Local\Temp\ipykernel_13116\2624616270.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data.append(twothousand)


In [423]:
#set the index to the month and transpose/rename before transposing back and resetting the index without dropping
data = data.set_index('month').transpose()
data = data.rename(columns = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
                              'May': 5, 'June': 6, 'July': 7, 'August': 8, 
                              'September': 9, 'October': 10, 'November': 11, 'December': 12, 
                             'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 
                              'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
                              'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})
data = data.transpose().reset_index(drop = False)

In [424]:
data['year'].unique()

array(['90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '00',
       '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'],
      dtype=object)

In [425]:
#do the same for year
data = data.set_index('year').transpose()
data = data.rename(columns = {'90': 1990, '91': 1991, '92': 1992, '93': 1993, '94': 1994, '95': 1995, 
                              '96': 1996, '97': 1997, '98': 1998, '99': 1999, 
                              '00': 2000, '1': 2001, '2': 2002, '3': 2003, '4': 2004, '5': 2005, 
                              '6': 2006, '7': 2007, '8': 2008, '9': 2009, '10': 2010, '11': 2011, 
                              '12': 2012, '13': 2013, '14': 2014, '15': 2015, '16': 2016, '17': 2017, 
                              '18': 2018, '19': 2019, '20': 2020, '21': 2021, '22': 2022})
data = data.transpose().reset_index(drop = False)

In [426]:
#add a filler for datetime to assume that this refers to the first day of each month
data['day'] = 1

In [427]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day
0,1990,1,LF:Labor Force,US,Jan-90,126027143.4,1
1,1990,1,LF:Employed,US,Jan-90,118985218.4,1


In [428]:
#set these to datatype integer
cols = ['month', 'year', 'day']
data[cols] = data[cols].astype(int)

In [429]:
#we only want years 2000 on
data = data.loc[data['year'] > 1999]

In [430]:
#create a time stamp with these
test['TimeStamp'] = pd.to_datetime(test[['year', 'month', 'day']])

In [431]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day
9120,2000,1,LF:Labor Force,US,January-00,143568554.6,1
9121,2000,1,LF:Employed,US,January-00,137572912.7,1


In [432]:
#drop the Time and day columns and check before exporting
data = data.drop(columns = ['Time', 'day'])
data.tail()

,year,month,Datapoint,NAME,Value
30091,2000,5,LF:Unemployed %,Williamson County,2.414991
30092,2000,5,LF:Labor Force,Wilson County,49733.12781
30093,2000,5,LF:Employed,Wilson County,48282.83375
30094,2000,5,LF:Unemployed,Wilson County,1546.71651
30095,2000,5,LF:Unemployed %,Wilson County,3.095518


In [435]:
#make sure the index is reset to save to feather format
data.reset_index(drop = True, inplace = True)

In [438]:
#export to feather
data.to_feather('../../Outputs/JOBSEQ_MonthlyData')